In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras import layers, initializers
from sklearn import metrics

from data_processing.parse_data import get_tracks_data
from data_processing.cluster_data import create_clusters
from analyse import validation
from data_processing.parse_data import *

c:\Users\jama2\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv(r'params_of_tracks_candidates.txt', sep=", ", engine='python')

In [5]:
df['good'] = df['duplicate'] == df['fake']
df['good'].astype('int8')

0          1
1          0
2          0
3          1
4          0
          ..
5989402    1
5989403    0
5989404    0
5989405    1
5989406    0
Name: good, Length: 5989407, dtype: int8

In [6]:
indexes = df['prototrackIndex']

In [7]:
event_number = df['#format: eventNumber']

In [8]:
df.drop(labels=['#format: eventNumber', 'prototrackIndex'], axis=1, inplace=True)

In [17]:
df.columns

Index(['nMeasurements', 'nStates', 'nOutliers', 'nHoles', 'nSharedHits',
       'chi2PerNDF', 'pT', 'eta', 'duplicate', 'fake', 'good'],
      dtype='object')

In [75]:
y = df['good']
y = y.astype('float32')
X = df.iloc[:, :-3]

In [76]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [77]:
X.shape

(5989407, 8)

In [78]:
import tensorflow as tf

In [85]:
class margin_ranking_loss(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
  def call(self, y_true, y_pred):
    margin = 0.05
    return tf.reduce_mean((tf.maximum(0.0, y_true - y_pred + margin)))

In [86]:
def create_model():
    model = Sequential()
    model.add(Dense(8, input_dim=len(X[0, :]), activation='relu',
                    kernel_initializer='random_normal'))
    model.add(Dense(10,activation='relu',kernel_initializer='random_normal'))
    model.add(Dense(15,activation='relu',kernel_initializer='random_normal'))
    model.add(Dense(10,activation='relu',kernel_initializer='random_normal'))
    model.add(Dense(1,activation='sigmoid',
                    kernel_initializer='random_normal'))
    model.compile(loss=margin_ranking_loss(),
                optimizer = 'adam',
                metrics =['accuracy'])
    
    return model


In [87]:
model = create_model()

In [88]:
checkpoint_path = "cp.ckpt"

In [89]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [90]:
model.fit(X, y, validation_split = 0.25, batch_size =64,
          callbacks=[cp_callback],verbose=2,epochs=5)

Epoch 1/5

Epoch 1: saving model to cp.ckpt
70189/70189 - 101s - loss: 0.0169 - accuracy: 0.3324 - val_loss: 0.0166 - val_accuracy: 0.3326 - 101s/epoch - 1ms/step
Epoch 2/5

Epoch 2: saving model to cp.ckpt
70189/70189 - 110s - loss: 0.0166 - accuracy: 0.3324 - val_loss: 0.0166 - val_accuracy: 0.3326 - 110s/epoch - 2ms/step
Epoch 3/5

Epoch 3: saving model to cp.ckpt
70189/70189 - 109s - loss: 0.0166 - accuracy: 0.3324 - val_loss: 0.0166 - val_accuracy: 0.3326 - 109s/epoch - 2ms/step
Epoch 4/5

Epoch 4: saving model to cp.ckpt
70189/70189 - 104s - loss: 0.0166 - accuracy: 0.3324 - val_loss: 0.0166 - val_accuracy: 0.3326 - 104s/epoch - 1ms/step
Epoch 5/5

Epoch 5: saving model to cp.ckpt
70189/70189 - 101s - loss: 0.0166 - accuracy: 0.3324 - val_loss: 0.0166 - val_accuracy: 0.3326 - 101s/epoch - 1ms/step


In [91]:
model.load_weights(checkpoint_path)

In [92]:
loss, acc = model.evaluate(X, y, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

187169/187169 - 169s - loss: 0.0166 - accuracy: 0.3325 - 169s/epoch - 905us/step
Restored model, accuracy: 33.25%


In [94]:
preds = model.predict(X)
preds


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [10]:
model = Sequential()
model.add(Dense(8, input_dim=len(X[0, :]), activation='relu',
                kernel_initializer='random_normal'))
model.add(Dense(10,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(15,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(10,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(1,activation='sigmoid',
                kernel_initializer='random_normal'))
model.compile(loss=margin_ranking_loss(),
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics =['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,
                        verbose=1, mode='auto', restore_best_weights=True)

model.fit(X, y, validation_split = 0.25, batch_size =64,
          callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
6840/6840 - 63s - loss: 1.0065 - accuracy: 0.9671 - val_loss: 1.0000 - val_accuracy: 0.9667 - 63s/epoch - 9ms/step
Epoch 2/1000
6840/6840 - 56s - loss: 1.0000 - accuracy: 0.9671 - val_loss: 1.0000 - val_accuracy: 0.9667 - 56s/epoch - 8ms/step
Epoch 3/1000
6840/6840 - 53s - loss: 1.0000 - accuracy: 0.9671 - val_loss: 1.0000 - val_accuracy: 0.9667 - 53s/epoch - 8ms/step
Epoch 4/1000
6840/6840 - 51s - loss: 1.0000 - accuracy: 0.9671 - val_loss: 1.0000 - val_accuracy: 0.9667 - 51s/epoch - 7ms/step
Epoch 5/1000
6840/6840 - 52s - loss: 1.0000 - accuracy: 0.9671 - val_loss: 1.0000 - val_accuracy: 0.9667 - 52s/epoch - 8ms/step
Epoch 6/1000
Restoring model weights from the end of the best epoch: 1.
6840/6840 - 49s - loss: 1.0000 - accuracy: 0.9671 - val_loss: 1.0000 - val_accuracy: 0.9667 - 49s/epoch - 7ms/step
Epoch 6: early stopping


In [21]:
# to save a model

model.save('nn-model')

INFO:tensorflow:Assets written to: nn-model\assets


INFO:tensorflow:Assets written to: nn-model\assets


In [22]:
preds = model.predict(X)

17099/17099 [==============================] - 105s 6ms/step


In [23]:
preds.shape

(547156, 1)

In [24]:
df_preds = pd.DataFrame(columns=['track_id', 'pred'])
df_preds['event'] = event_number
df_preds['track_id'] = indexes
df_preds['pred'] = preds
df_preds['new'] = df_preds.apply(lambda x: str(int(x.event)) + '/' + str(int(x.track_id)) , axis=1)
dct_preds = dict(zip(df_preds['new'], df_preds['pred']))

In [25]:
import json

with open('predictions.json', 'w', encoding='utf-8') as file:
    json.dump(dct_preds, file)

In [28]:
def process_event(event_num, dct_preds, hits=3):
    track_list = get_tracks_data(f"../data/event_{event_num}_prototracks.txt", f"../data/event_{event_num}_space_points.txt", True)
    clusters = create_clusters(track_list, min_n_shared_hits=hits)
    event_curr_good = []
    for cluster in clusters:
        max_score = 0
        best_track = None
        for dct in cluster:
            track_id = [*dct][0]
            hits = dct[track_id]
            if dct_preds[f'{event_num}/{track_id}'] > max_score:
                max_score = dct_preds[f'{event_num}/{track_id}']
                best_track = hits
        event_curr_good.append(best_track)
    track_dict = get_hits_data(f"../data/event_{event_num}_space_points.txt")
    hit_list = get_hits_data_for_validation(f"../data/event_{event_num}_space_points.txt")
    track_id_dict = get_track_id(f"../data/event_{event_num}_trackIds.txt")

    characteristic_dict = validation.calc_characteristics(event_curr_good, hit_list, track_dict, track_id_dict)
    
    return characteristic_dict
    
    

In [29]:
print(process_event(3, dct_preds, hits=3))

{'efficiency': 0.9230769230769231, 'fake_rate': 0.0, 'duplication_rate': 0.0, 'purity': 0.39473684210526316, 'num_recognize_track': 12, 'num_real_track': 13, 'num_duplicate_track': 0, 'num_proto_track': 38, 'num_fake_track': 0, 'num_reco_dupl_track': 15}


In [21]:
df_result_1 = pd.DataFrame(columns=['efficiency', 'fake_rate', 'duplication_rate', 'purity',
                                    'num_recognize_track', 'num_real_track', 'num_duplicate_track',
                                    'num_proto_track', 'num_fake_track', 'num_reco_dupl_track'])
df_result_3 = pd.DataFrame(columns=['efficiency', 'fake_rate', 'duplication_rate', 'purity',
                                    'num_recognize_track', 'num_real_track', 'num_duplicate_track',
                                    'num_proto_track', 'num_fake_track', 'num_reco_dupl_track'])
df_result_5 = pd.DataFrame(columns=['efficiency', 'fake_rate', 'duplication_rate', 'purity',
                                    'num_recognize_track', 'num_real_track', 'num_duplicate_track',
                                    'num_proto_track', 'num_fake_track', 'num_reco_dupl_track'])
df_result_7 = pd.DataFrame(columns=['efficiency', 'fake_rate', 'duplication_rate', 'purity',
                                    'num_recognize_track', 'num_real_track', 'num_duplicate_track',
                                    'num_proto_track', 'num_fake_track', 'num_reco_dupl_track'])
df_result_9 = pd.DataFrame(columns=['efficiency', 'fake_rate', 'duplication_rate', 'purity',
                                    'num_recognize_track', 'num_real_track', 'num_duplicate_track',
                                    'num_proto_track', 'num_fake_track', 'num_reco_dupl_track'])

In [22]:
for i in range(101):
    print(f'------{i}------')
    for df, hit in zip([df_result_1, 
                        df_result_3,
                        df_result_5,
                        df_result_7,
                        df_result_9],
                        [1, 3, 5, 7, 9]):
        dct = process_event(i, dct_preds, hit)
        eff = dct['efficiency']
        fake = dct['fake_rate']
        dup = dct['duplication_rate']
        pur = dct['purity']
        reco = dct['num_recognize_track']
        real = dct['num_real_track']
        dupl = dct['num_duplicate_track']
        proto = dct['num_proto_track']
        fake = dct['num_fake_track']
        reco_dupl = dct['num_reco_dupl_track']
        
        df.loc[i] = [eff, fake, dup, pur, reco, real, dupl, proto, fake, reco_dupl]

------0------
------1------
------2------
------3------
------4------
------5------
------6------
------7------
------8------
------9------
------10------
------11------
------12------
------13------
------14------
------15------
------16------
------17------
------18------
------19------
------20------
------21------
------22------
------23------
------24------
------25------
------26------
------27------
------28------
------29------
------30------
------31------
------32------
------33------
------34------
------35------
------36------
------37------
------38------
------39------
------40------
------41------
------42------
------43------
------44------
------45------
------46------
------47------
------48------
------49------
------50------
------51------
------52------
------53------
------54------
------55------
------56------
------57------
------58------
------59------
------60------
------61------
------62------
------63------
------64------
------65------
------66------
-----

In [60]:
for df, hit in zip([df_result_1, df_result_3,
                    df_result_5, df_result_7, df_result_9],
                    [1, 3, 5, 7, 9]):
        df.to_csv(f'result_{hit}.csv', index=False)